# Wie genau funktioniert Extractive Question Answering mit Huggingface und BERT?

In dem einführenden Notebooks zu Transformern (*01_Transformer_Intro*) haben sie gesehen, welche vielfältigen Aufgaben moderne Transformerarchitekturen lösen können. Weiter haben wir ihnen gezeigt, wie sie mit dem Huggigface __transformers__ Paket verschiedene, bereits auf konkrete Aufgaben nachtrainierte __pipelines__ aufrufen und benutzen können.

In diesem Notebook wollen wir ihnen zeigen, wie sie ein bestimmtes, [vor- und bereits auf einer Frage-Antwort-Aufgabe nachtrainiertes BERT-Modell](https://huggingface.co/Sahajtomar/GBERTQnA) benutzen können, um deutsche Fragen anhand deutscher Texte zu beantworten.

## Überblick über die Aufgaben in diesem Notebook

- __Lesen sie__ die einführenden __Texte__, die die grundlegende Funktionsweise eine Frage-Antwort-Systems auf der grundlage eines fertig nachtrainierten __englischen Frage-Antwort-Modells demonstrieren__. 
    - __Führen sie__ dabei auch die zugehörigen __Codezellen aus__ und versuchen sie die Logik des Codes und die Ausgaben nachzuvollziehen und 
    - __bearbeiten sie die hin und wieder eingestreuten Übungsaufgaben__

- __Schreiben sie im Anschluss eigenen Code__ in der letzten Codezelle, um ganz analog mit einem fertig nachtrainierten Transformermodell __deutsche Fragen anhand deutscher Informationstexte ("Kontexte") zu beantworten__.

- __Testen und evaluieren sie kritisch__ die Leistungsfähigkeit des __nachtrainierten deutschen Frage-Antwortmodells__. Sammeln sie in dem entsprechenden Miro-Board, welches in den geteilten Notizen verlinkt ist, wieder Fragen, Kontexte, und Antworten, für die das Modell 
    - __Erstaunlich oder beeindruckend gut__ funktioniert hat.
    - __Überraschend schlecht__ funktioniert hat.
    - Oder Beispiele, bei denen sie den Output des Netzwerkes auf andere Weise __bemerkenswert oder witzig__ finden.
    
- Sammeln sie in dem __Miro-Board__ weiterhin für die einzelnen Beispiele (ihre eigenen aber auch gerne für die ihrer Kommiliton*innen) __Ideen zu den folgenden Themen__:
    - __Woran__ die entsprechende Performance des Netzwerks liegen könnte.
    - Auf __welcher Art von Daten__ man eventuell __nachtrainieren__ könnte, um die schlechte Performance auf Negativbeispielen zu Verbessern.
    - Wie man die __Performance__ eventuell __verbessern__ könnte, in dem man zusätzlichen Code schreibt, der die __Netzwerk In- und Outputs anders vor bzw. nachverarbeitet__.

Erinnern wir uns zunächst an das Beispiel zur __"Question-Answering"__ Pipeline aus dem vorhergehenden Notebook.

In [ ]:
from transformers import pipeline

nlp = pipeline("question-answering")

# Als "Kontext" wird ein Text bezeichnet, aus dem die Antwort auf eine gegebene Frage extrahiert werden soll.
# Häufig ist die Aufgabe, die Antwort auf eine Frage direkt aus diesem Kontext herauszuschneiden
# bzw. mit "Textmarker" anzustreichen. Damit ist gemeint, die Position der Antwort innerhalb
# des Kontextes auszugeben.

# Experimentieren sie mit Unterschiedlichen Textabschnitten und dazu (mehr oder weniger gut)
# passenden Fragen.
context = """Extractive Question Answering is the task of extracting an answer from a text given a question. 
An example of a question answering dataset is the SQuAD dataset, which is entirely based on that task. 
If you would like to fine-tune a model on a SQuAD task, you may leverage the 
examples/question-answering/run_squad.py script from the Huggingface transformer repository.
"""

frage1 = "What is extractive question answering?"
frage2 = "What is a good example of a question answering dataset?"

# Frage 1 anhand des gegebenen Kontexts beantworten
output_frage1 = nlp(question=frage1, context=context)

# Antwort auf Frage 1 ausgeben
print('Frage: ' + frage1)
print('Antwort: ' + output_frage1['answer'])
print('Gefunden im Kontext von Position %d bis Position %d!' % (output_frage1['start'], output_frage1['end']))
print('Score: %f' % output_frage1['score'])
print()

# Frage 2 anhand des gegebenen Kontexts beantworten
output_frage2 = nlp(question=frage2, context=context)

# Antwort auf Frage 2 ausgeben
print('Frage: ' + frage2)
print('Antwort: ' + output_frage2['answer'])
print('Gefunden im Kontext von Position %d bis Position %d!' % (output_frage2['start'], output_frage2['end']))
print('Score: %f' % output_frage2['score'])
print()

**Aufgabe**: Falls sie es im letzten Notebook noch nicht getan haben, experimentieren sie etwas, in dem sie verschiedene Texte (z.B. aus der englischen Wikipedia) und verschiedene Fragen ausprobieren und den Output betrachten.

## Geht das auch mit meinen eigenen Modellen? Und vielleicht sogar auf Deutsch?

Die "Pipelines", die Huggingface bereitstellt, sind sehr einfach zu benutzen und funktionieren für die entsprechenden englischen Standardaufgaben schon recht gut. Damit man jedoch eigene, z.B. auf eigenen Trainingsdaten nachtrainierte, Transformernetzwerke benutzen kann, muss man ein *wenig* (wirklich nicht sehr viel) mehr Programmieraufwand investieren. Zunächst werden wir jedoch erst noch ein paar Begrifflichkeiten erklären.

### Embeddings, Tokenizer und Modelle

Streng genommen braucht man für jede Anwendung, die mit natürlichen Texten arbeitet, nicht nur ein trainiertes neuronales Netzwerk, sondern auch eine Funktion, die die Eingabetexte in ein Format übersetzt, die das neuronale Netzwerk verarbeiten kann, und die Netzwerk-Outputs wieder in lesbaren Text zurückübersetzt. Neuronale Netze arbeiten hier oft mit "Embeddings", die verschiedenen sogenannte "Tokens" der Rohdaten (das können Buchstaben, Worte, oder aber auch Silben bzw. Wortteile sein) bestimmte Aktivierungsmuster in einem neuronalen "Input-Layer" zuordnen. Diese Token-spezifischen Aktivierungsmuster können fest vorgegeben sein. Ein häufiges Beispiel hierfür ist das sogenannte "One-Hot-Embedding", bei dem jeweils ein Neuron im Input-Layer ein bestimmtes Token repräsentiert. Immer mehr Architekturen gehen jedoch dazu über, die Embeddings der einzelnen Tokens zu lernen. Dies erlaubt es, schon während des Embeddings bestimmte Zusammenhänge abzubilden, in dem ähnliche Tokens ähnlichen Aktivierungsmustern zugewiesen werden.

Bevor der "Embedding"-Schritt jedoch stattfinden kann, muss der Eingabetext zunächst in einzelne "Tokens" zerlegt werden. Eine der einfachsten Vorgehensweisen dabei ist es, direkt einzelne Buchstaben bzw. Zeichen des Eingabetextes als Tokens zu verwenden. Dies hat jedoch den Nachteil, dass die neuronalen Netzwerke schon für kürzere Textabschnitte *sehr viele* Tokens verarbeiten müssen. D.h. man stößt hierbei sehr schnell an Grenzen bezüglich der maximalen Länge von Sequenzen, die bei gegebenem Arbeitsspeicher auf einmal verarbeitet werden können. Eine andere Option wäre, einzelne Worte als Tokens zu benutzen. Dies funktioniert gut in Sprachen, in denen wenig konjugiert oder dekliniert wird. In Sprachen wie dem Deutschen muss man sich jedoch überlegen, wie man deklinierte bzw. konjugierte Formen behandelt. Die BERT-Modelle, mit denen wir heute arbeiten wollen, benutzen als datengetriebene Lösung dieses Problems den so genannte _WordPiece_ Tokenizer. Dieser datengetriebene Algorithmus erstellt ein Vokabular von Tokens anhand eines vorgegebenen Textkorpus. Man kann dem Algorithmus zusätzlich die gewünschte Länge des erzeugte "Wortschatzes" an Tokens, sowie (optional) eine Liste von "Starttokens" vorgegeben, die auf jeden Fall im Wortschatz des Tokenizers enthalten sein sollen. Der WordPiece Algorithmus fügt dann zu dem Startalphabet so lange Wortteile oder ganze Wörter hinzu, bis die gewüschte Größe des Alphabets erreicht ist. Dabei werden die zum Wortschatz hinzugefügten Tokens so ausgewählt, dass die Zerlegung des vorgegebenen Textkorpus mittels des Token-Wortschatzes möglichst kurz wird, d.h. so dass man den vorgegebenen Korpus durch eine möglichst kurze Sequenz an Tokens repräsentieren kann. Im Fall von BERT wurde der Wortschatz mit dem Alphabet der einzelnen Buchstaben und Zeichen, die in dem Textkorpus vorkamen, initialisiert. Die hinzugefügten Tokens bestehen zum großen Teil aus Worten, aber auch aus Silben und Wortteilen. Dieser Tokenizer wurde beim Training der Netzwerke benutzt, um Eingabesätze in entsprechende Sequenzen von Tokens aus dem Wortschatz des Tokenizers umzuwandeln, wobei jedem Token eine feste Zahl (ein Index) zugewesen wurde. Das BERT-Netzwerk hat dann die entsprechende Einbettung dieser Token-Indizes als Aktivierungen seines Input-Layers gelernt. 

Dies bedeutet aber, dass das BERT-Netzwerk nie direkt die einzelnen Zeichenketten der Tokens gesehen hat, sondern immer nur die entsprechenden Zahlen-Indizes. D.h. um einen neuen Text mit einem trainierten BERT-Netzwerk zu verarbeiten, braucht man nicht nur das trainierte BERT-Netzwerk, sondern auch **den selben Tokenizer**, der auch im Training benutzt wurde, um die richtige Übersetzung zwischen geschriebenem Text und Input-Token-Indizes zu gewährleisten.

Wenn man deshalb mit selbst-trainierten und ganz bestimmten BERT-Modellen arbeiten möchte, reicht es deshalb nicht, nur diese Modelle zu laden, sondern man muss auch die richtigen Tokenizer benutzen.

In der folgenden Zelle werden wir deshalb zunächst das Modell, welches wir für das erste Beispiel benutzen wollen (https://huggingface.co/bert-large-uncased-whole-word-masking-finetuned-squad), **und den dazugehörigen Tokenizer** laden. Diese Schritte wurden bis jetzt immer von der entsprechenden __Pipeline__ automatisch gehandhabt.

In [ ]:
# AutoTokenizer und AutoModelForQuestionAnswering
# erlauben es, die entsprechenden Tokenizer und Modelle 
# aus dem Huggingface.co - Repository zu laden.
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

Zunächst schauen wir uns an, wie der Tokenizer genau funktioniert. Dazu wandeln wir eine Zeichenkette in die entsprechenden Tokens um.

In [ ]:
# Wir definieren einen Text, den wir in eine Liste von Tokens umwandeln wollen.
text = """ Alan Mathison Turing OBE FRS (/ˈtjʊərɪŋ/; 23 June 1912 – 7 June 1954) was an English mathematician, 
computer scientist, logician, philosopher, and theoretical biologist. In 1938, he obtained his PhD from the
Department of Mathematics at Princeton University. During the Second World War, Turing worked for the 
Government Code and Cypher School (GC&CS) at Bletchley Park. Here, he devised a number of techniques 
for speeding the breaking of German ciphers. Turing played a crucial role in cracking intercepted coded messages 
that enabled the Allies to defeat the Nazis in many crucial engagements, including the Battle of the Atlantic.
Due to the problems of counterfactual history, it is hard to estimate the precise effect Ultra intelligence 
had on the war, but Professor Jack Copeland has estimated that this work shortened the war in Europe by 
more than two years and saved over 14 million lives.
After the war, Turing worked at the National Physical Laboratory, where he designed the Automatic Computing Engine. 
The Automatic Computing Engine was one of the first designs for a stored-program computer."""

text_tokens = tokenizer.tokenize(text)

Betrachten wir die generierte Liste von Output-Tokens:

In [ ]:
print(text_tokens)

print('Länge der Token-Sequenz:')
print(len(text_tokens))
print('')

Wir sehen, dass viele der englischen Wörter in diesem Text, z.B. "english", "mathematician", "computer", "scientist", "biogist" einzelnen Tokens entsprechen.

Wir sehen aber auch, dass einige der Sonderzeichen, z.B. die Klammern oder der Bindestrich, als einzelne Tokens abgebildet werden. 

Einige Tokens beginnen mit einem doppelten Hashtag "##". Das bedeutet, dass diese Tokens beim Zerlegen **innerhalb** eines Wortes gefunden wurden. Solche Tokens werden von den entsprechenden Gegenstücken, die alleine oder am Anfang eines Wortes stehen, unterschieden. Diese Unterscheidung erlaubt es, Wortgrenzen direkt in der Sequenz der Tokens zu repräsentieren, ohne eigene "Trennzeichen" zu benutzen (z.B. Leerzeichen). Dies verringert wiederum die Anzahl an Tokens, die auf einmal verarbeitet werden müssen, um einen bestimmten Text mit dem Transformer-Netzwerk zu verarbeiten. 

Weitere Informationen zum spezifischen WordPiece-Algorithmus, der benutzt wurde um den Tokenizer zu erstellen finden sie hier: https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/#22-tokenization

Die Darstellung der Zeichenkette als Liste von "Tokens", die die einzelnen Zeichenketten enthält, dient jedoch eher der Illustration. Worauf die Netzwerke - wie bereits erwähnt - trainiert wurden, sind Sequenzen von Indizes, also ganzen Zahlen die die einzelnen Tokens identifizieren.

In der folgenden Zelle sehen wir, wie wir in der Praxis aus einem Text eine solche Index-Liste erstellen

In [ ]:
id_tokens = tokenizer.convert_tokens_to_ids(text_tokens)

print('Ids:')
print(id_tokens)

Die Anzahl der Tokens und die Übersetzungstabelle zwischen Text-Token und Index kann man wie folgt einsehen:

In [ ]:
print('Anzahl der verschiedenen Tokens:')
print(tokenizer.vocab_size)

In [ ]:
print('Übersetzungstabelle:')
print(tokenizer.vocab)

# Inputformat für Question-Answering

Um eine Frage und einen Kontext, aus dem die Antwort auf diese Frage extrahiert werden soll, an ein entsprechend nachtrainiertes Transformermodell zu schicken, muss man bestimmte Formatkonventionen einhalten.

Wir benutzen fast nur Modelle, die auf Grundlage einer [BERT-Architektur](http://jalammar.github.io/illustrated-bert/) trainiert wurden. Diese Modelle erwarten den Input (fast) immer in folgendem Format:

**\[CLS\]** Textblock 1 **\[SEP\]** Textblock 2 **\[SEP\]**

Hierbei sind **\[CLS\]** und **\[SEP\]** besondere Tokens, die die Indizes **101** ( **'\[CLS\]'** ) bzw. **102** ( **'\[SEP\]'** ) besitzen. 

__Aufgabe:__ Schauen sie gerne einmal mit der Suchfunktion ihres Browsers, ob sie die entsprechenden Einträge in der Übersetzungstabelle oben finden und vergewissern sie sich so, dass diese Zuordnung auch in dem von uns genutzten Modell zutrifft. 

Oft werden die Inputs dann auch noch mit Padding-Tokens ( **\[PAD\]** , Index **0** ) aufgefüllt, bis die maximale Länge der Tokenkette erreicht ist, die das Transformernetzwerk auf einmal verarbeiten kann. Letzteres dient vor allem dazu, dass während des Trainings, wo man zur Berechnung des Gradienten der Zielfunktion keine einzelnen Sequenzen, sondern ganze Mini-Batches benutzt, alle Zeichenketten eines Mini-Batches die selbe (nämlich die maximale) Länge besitzen. Dies erleichtert die gleichzeitige, parallele Verarbeitung aller Zeichenketten eines Minibatches auf entsprechender Hardware, z.B. Graphikkarten.

Wir können auch ohne das gesamte Vokabular ausgeben zu müssen überprüfen, ob der von uns geladene Tokenizer diese wichtigen, besonderen Tokens auf die entsprechenden Indizes abbildet, in dem wir die entsprechenden Einträge in der tokenizer.vocab-Struktur (einem Python __dictionary__) ausgeben:

In [ ]:
print('[CLS] Token:')
print(tokenizer.vocab['[CLS]'])

In [ ]:
print('[SEP] Token:')
print(tokenizer.vocab['[SEP]'])

In [ ]:
print('[PAD] Token:')
print(tokenizer.vocab['[PAD]'])

Im Spezialfall eines Transformer-Modelles, welches auf Question-Answering fingetuned wurde, wird der Input in folgendem Format erwartet:
    
**\[CLS\]** Frage **\[SEP\]** Kontext, aus dem die Antwort extrahiert werden soll **\[SEP\]**

Um so einen Input zu erzeugen, können wir den Tokenizer direkt mit folgenden Parametern aufrufen:

In [ ]:
frage = "Where did Alan Turing work during the Second World War?"

text = """ Alan Mathison Turing OBE FRS (/ˈtjʊərɪŋ/; 23 June 1912 – 7 June 1954) was an English mathematician, 
computer scientist, logician, philosopher, and theoretical biologist. In 1938, he obtained his PhD from the
Department of Mathematics at Princeton University. During the Second World War, Turing worked for the 
Government Code and Cypher School (GC&CS) at Bletchley Park. Here, he devised a number of techniques 
for speeding the breaking of German ciphers. Turing played a crucial role in cracking intercepted coded messages 
that enabled the Allies to defeat the Nazis in many crucial engagements, including the Battle of the Atlantic.
Due to the problems of counterfactual history, it is hard to estimate the precise effect Ultra intelligence 
had on the war, but Professor Jack Copeland has estimated that this work shortened the war in Europe by 
more than two years and saved over 14 million lives.
After the war, Turing worked at the National Physical Laboratory, where he designed the Automatic Computing Engine. 
The Automatic Computing Engine was one of the first designs for a stored-program computer."""

inputs = tokenizer(frage,
                   text,
                   add_special_tokens = True, # Fügt die CLS und SEP Tokens an den
                                              # entsprechenden Stellen ein
                   return_tensors = "pt", # Gibt den Input gleich in einem Format aus, 
                                         # welches PyTorch weiterverarbeiten kann
                   padding=True  # Füllt die Token-Liste mit [PAD]
                                            # Tokens bis zur Maximallänge auf                   
                  )

## Berechnung des Outputs des Netzwerkmodells
Die so vorverarbeiteten Inputs können wir nun dem für Frage-Antwort-Aufgaben nachtrainierten BERT-Modell übergeben.

In [ ]:
output = model(**inputs)

## Netzwerk-Outputs und deren Verarbeitung

Als nächstes Betrachten wir den Output, den uns das auf Question-Answering nachtrainierte BERT-Modell zurückgibt.

In [ ]:
print('output:')
print(output)

Wir sehen, dass die Output-Struktur zwei lange Listen, **start_logits** und **end_logits** enthält. Diese Zahlenwerte kodieren die vom Modell vorhergesagten Wahrscheinlichkeiten, dass die Antwort auf die Frage an der entsprechenden Position (d.h. bei dem entsprechenden Eintrag der Input-Tokenliste) beginnt (**start_logits**) bzw endet (**end_logits**). Um genau zu sein handelt es sich um **logits**, d.h. um die  **Logarithmen** der (noch nicht normierten) vorhergesagten Wahrscheinlichkeiten. Um diese in entsprechende Wahrscheinlichkeiten zu übersetzen kann man, wie wir es bereits bei den RNNs gelernt haben, die __Softmax__-Funktion benutzen. Dieser Schritt ist jedoch **nicht** notwendig, wenn man sich nur für das Token mit der  maximalen Wahrscheinlichkeit interessiert, da die Softmax-Funktion immer dem Token mit dem höchsten logit-Wert auch die höchste Wahrscheinlichkeit zuordnet.

D.h. um eine Antwort zu erzeugen, können wir zum Beispiel den Text extrahieren, der bei dem Token mit dem maximalen Wert von **start_logits** beginnt (d.h.  bei dem Token, das die maximale Wahrscheinlichkeit dafür besitzt, dass die Antwort hier startet) und der bei dem Token mit dem maximalen Wert von **end_logits** endet (d.h. bei dem Token, das die maximale Wahrscheinlichkeit besitzt, dass die Antwort hier endet).

Wir extrahieren die Position dieser Tokens mit der **torch.argmax** Funktion, die eine Liste von Zahlenwerten als Input erthält, und den Index zurückgibt, an dem sich der Eintrag mit dem höchsten Wert befindet.

In [ ]:
import torch

start_index = torch.argmax(output['start_logits'])
print('Wahrscheinlichste Startposition der Antwort:')
print(start_index)

In [ ]:
end_index = torch.argmax(output['end_logits'])
print('Wahrscheinlichste Endposition der Antwort:')
print(end_index)

Um die Antwort zu erhalten, extrahieren wir jetzt die entsprechenden Tokens aus der Input-Token-Sequenz und übersetzen sie zurück in normalen Text.

Dazu Wandeln wir die vom Tokenizer generierte Sequenz der Input-Token-Indices zunächst in eine Liste um.

In [ ]:
# Input in Liste von Token-Indizes umwandeln
input_ids = inputs['input_ids'].tolist()[0]

# Gesamte Input-Token-Index-Liste Ausgabe
print('Gesamte Input-Token-Index-Liste:')
print(input_ids)
print('')

### Einschub: Listen-Indizierung
Um auf einen bestimmten Teilabschnitt einer Liste zuzugreifen, kann man den Doppelpunkt-Operator benutzen. Dazu haben wir in den folgenden Codezellen dieses Unterabschnitts einen kurzen Einschub vorbereitet, der nichts mit dem restlichen Transformer-Code zu tun hat, sondern nur dazu dient, den Umgang mit Listen und Indizes etwas zu üben. Zunächst erstellen wir eine Liste, die verschiedenen Einträge enthält Also während z.B. 

In [ ]:
beispiel_liste = ['Das','ist','eine','kurze','Beispielliste', 'mit', 8, 'Elementen']

Beachten sie, dass Listen (__die durch eckige Klammern "[ ... ]" erzeugt werden__) aus __beliebigen Einträgen__ bestehen können, die __durch Kommata voneinander getrennt__ sind. Das können wie in unserem Beispiel Zeichenketten und eine Zahl sein, aber auch durchaus komplexere Datenstrukturen.

Die Länge einer Liste wird durch die Funktion __len__ zurückgegeben. Einzelne Elemente kann man durch einzelne Zahlen ("Indizes") auswählen. Hierbei ist zu beachten, dass der erste Eintrag der Liste den Index **0** erhält.

In [ ]:
# Gibt die Länge der Liste zurück:
len(beispiel_liste)

**Aufgabe:** Experimentieren sie in der folgenden Codezelle mit verschiedenenen Indizes, bis ihnen das Prinzip klar ist.

In [ ]:
# Gibt das entsprechende Element der Liste zurück
beispiel_liste[5]

Der Doppelpunkt **':'** Operator kann nun benutzt werden, um einen gesamten Teilbereich einer Liste auszuwählen. Dabei werden alle Einträge vom Index, der links des Doppelpunktes steht, bis zu dem Eintrag **vor** dem Index, der rechts des Doppelpunktes steht zurückgegeben.

**Aufgabe:** Experimentieren sie in der folgenden Codezelle mit den Indizeslinks und rechts des Doppelpunktes,
bis ihnen das Prinzip klar ist.

In [ ]:
# Gibt einen Teilbereich der Liste aus.
beispiel_liste[0:3]

## Zurück zum Transformer-Output

Mit diesem Hintergrundwissen über Listen-Indizierung können wir nun aus der Liste der Input-Token-Indices *alle Token-Indizes* vom *start_index* bis **einschließlich** des Tokens an der Position des *end_index* auswählen.

In [ ]:
# Extrahiere entsprechenden Bereich aus der 
# Token-Indizes-Liste
answer_token_ids = input_ids[start_index:end_index+1]

# Ausgabe
print('Antwort als Token-Index-Liste:')
print(answer_token_ids)

**Aufgabe**: Überlegen sie sich, warum es hinter dem Doppelpunkt "end_index**+1**" heißen muss.

Nun übersetzen wir die Tokens, die die Antwort darstellen, wieder zurück in eine normale Zeichenkette.

In [ ]:
# Indizes in Tokens übersetzen
answer_text_tokens = tokenizer.convert_ids_to_tokens(answer_token_ids)

print(answer_text_tokens)

Zuletzt konvertieren wir die Liste an Tokens noch in eine zusammenhängende Zeichenkette und geben diese aus.

In [ ]:
# Tokens zu Text zusammensetzen
complete_answer = tokenizer.convert_tokens_to_string(answer_text_tokens)

print(complete_answer)

***Juhu!*** Damit haben wir unsere Antwort.

## Zusammenfassung

Wenn man alles, was wir uns bis hierhin erarbeitet haben, in einer einzelnen Codezelle zusammenfasst, sind es tatsächlich nur ca. 30 Zeilen Code.

**Aufgabe**: Vollziehen sie nochmal die komplette Logik anhand des Codes in der folgenden Zelle nach. Experimentieren sie gerne auch noch ein wenig mit den Fragen- und Textinputs und beobachten und bewerten sie die entsprechenden Outputs.

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

frage = "Where did Alan Turing work during the Second World War?"

text = """ Alan Mathison Turing OBE FRS (/ˈtjʊərɪŋ/; 23 June 1912 – 7 June 1954) was an English mathematician, 
computer scientist, logician, philosopher, and theoretical biologist. In 1938, he obtained his PhD from the
Department of Mathematics at Princeton University. During the Second World War, Turing worked for the 
Government Code and Cypher School (GC&CS) at Bletchley Park. Here, he devised a number of techniques 
for speeding the breaking of German ciphers. Turing played a crucial role in cracking intercepted coded messages 
that enabled the Allies to defeat the Nazis in many crucial engagements, including the Battle of the Atlantic.
Due to the problems of counterfactual history, it is hard to estimate the precise effect Ultra intelligence 
had on the war, but Professor Jack Copeland has estimated that this work shortened the war in Europe by 
more than two years and saved over 14 million lives.
After the war, Turing worked at the National Physical Laboratory, where he designed the Automatic Computing Engine. 
The Automatic Computing Engine was one of the first designs for a stored-program computer."""

inputs = tokenizer(frage, text, add_special_tokens=True, return_tensors="pt", padding=True)
input_ids = inputs["input_ids"].tolist()[0]

output = model(**inputs)

start_index = torch.argmax(output['start_logits'])
end_index = torch.argmax(output['end_logits'])
    
antwort = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[start_index:end_index+1]))

print("Question: " + frage)
print("Answer: " + antwort)
print("")

### Geht das auch auf Deutsch?

Ja, dank https://huggingface.co/Sahajtomar/GBERTQnA.

Dabei handelt es sich um ein großes, auf einem riesigen Textkorpus vortrainiertes deutsches BERT Modell (https://huggingface.co/deepset/gbert-large), welches auf einem deutschen Frage-Antwortdatensatz (ca. 5000 Text-Frage-Antwort-Paare, dem deutschen Teil von https://github.com/facebookresearch/MLQA) nachtrainiert und auf einem weiteren deutschen Frage-Antwortdatensatz (ca. 1200 Text-Frage-Antwort-Paare, dem deutschen Teil von https://github.com/deepmind/xquad) validiert wurde.

Das Modell und den entsprechenden Tokenizer kann man wie in der folgenden Zelle laden:

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# Lade und initialisiere das deutsche Frage-Antwort-Modell https://huggingface.co/Sahajtomar/GBERTQnA.
tokenizer = AutoTokenizer.from_pretrained("Sahajtomar/GBERTQnA")
model = AutoModelForQuestionAnswering.from_pretrained("Sahajtomar/GBERTQnA")

Mit diesem auf Frage-Antwort-Paaren nachtrainierten deutschen BERT-Modell und dem dazugehörigen Tokenizer können wir jetzt auch deutsche Text-Frage-Paare verarbeiten. Einen Vorschlag für ein mögliches Text-Frage paar finden sie in der nächsten Zelle.

In [ ]:
text = """Der Pathologe Thomas Hodgkin präsentierte 1832 in London erstmalig den klinischen Verlauf 
sowie die Obduktionsbefunde von mehreren Patienten mit großen Lymphomen und Splenomegalie. 
Die pathologisch veränderten Lymphknoten waren entlang der großen Gefäße an Hals, Thorax 
und Abdomen zu finden. Später wurden charakteristische mehrkernige Riesenzellen gefunden, 
die heute als Sternberg-Reed-Zellen bezeichnet werden.
"""

frage = "Wo wurden veränderte Lymphknoten gefunden?"

## Programmieraufgabe
Schreiben sie mit Hilfe der Codebeispiele, die wir in diesem Notebook erarbeitet haben, in der folgenden Zelle den notwendigen Code, um die gegebene deutsche Frage mit Hilfe des gegebenen deutschen Textes zu beantworten und die Antwort auszugeben.

In [ ]:
# Fügen sie hier ihren Code ein


## Experimentieraufgabe

- __Testen und evaluieren sie kritisch__ die Leistungsfähigkeit des __nachtrainierten deutschen Frage-Antwortmodells__, in dem sie mit verschiedenen deutschen Fragen und Kontexten experimentieren. Sammeln sie in dem entsprechenden Miro-Board, welches in den geteilten Notizen verlinkt ist, wieder Fragen, Kontexte, und Antworten, für die das Modell 
    - __Erstaunlich oder beeindruckend gut__ funktioniert hat.
    - __Überraschend schlecht__ funktioniert hat.
    - Oder Beispiele, bei denen sie den Output des Netzwerkes auf andere Weise __bemerkenswert oder witzig__ finden.
- Sammeln sie in dem __Miro-Board__ für die einzelnen Beispiele (ihre eigenen aber auch gerne für die ihrer Kommiliton*innen) __Ideen zu den folgenden Themen__:
    - __Woran__ die entsprechende Performance des Netzwerks liegen könnte.
    - Auf __welcher Art von Daten__ man eventuell __nachtrainieren__ könnte, um die schlechte Performance auf Negativbeispielen zu Verbessern.
    - Wie man die __Performance__ eventuell __verbessern__ könnte, in dem man zusätzlichen Code schreibt, der die __Netzwerk In- und Outputs anders vor bzw. nachverarbeitet__.

<details><summary>Klicken sie <b>hier</b> für eine mögliche Musterlösung der Programmieraufgabe</summary>
<p>

```python

import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("Sahajtomar/GBERTQnA")
model = AutoModelForQuestionAnswering.from_pretrained("Sahajtomar/GBERTQnA")

text = """Der Pathologe Thomas Hodgkin präsentierte 1832 in London erstmalig den klinischen Verlauf 
sowie die Obduktionsbefunde von mehreren Patienten mit großen Lymphomen und Splenomegalie. 
Die pathologisch veränderten Lymphknoten waren entlang der großen Gefäße an Hals, Thorax 
und Abdomen zu finden. Später wurden charakteristische mehrkernige Riesenzellen gefunden, 
die heute als Sternberg-Reed-Zellen bezeichnet werden.
"""

frage = "Wo wurden veränderte Lymphknoten gefunden?"

inputs = tokenizer(frage, text, add_special_tokens=True, return_tensors="pt", padding=True)
input_ids = inputs["input_ids"].tolist()[0]

output = model(**inputs)

start_index = torch.argmax(output['start_logits'])
end_index = torch.argmax(output['end_logits'])

antwort = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[start_index:end_index+1]))

print("Frage: " + frage)
print("Antwort: " + antwort)
print("")

```

</p>
</details>